In [1]:
import joblib
from src import Utils, EnsembleFunctions
import pandas as pd
import numpy as np

KeyboardInterrupt: 

In [ ]:
import torch
torch.cuda.empty_cache()
torch.cuda.is_available = lambda: False

In [ ]:
Utils.seed_random_number_generators()

Random number generators seeded.


In [ ]:
def load_model(model_name: str):
  try:
    return joblib.load(model_name)
  except FileNotFoundError:
    print("ERROR: Model not found")
    return None

learner_bayes = load_model('model_bayes/Bayes.pkl')
learner_lstm = load_model('model_lstm/LSTM.pkl')
learner_bert = load_model('model_bert/mBERT.pkl')

/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_md' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)
/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_lg' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn

In [ ]:
learner_bayes

Pipeline(steps=[('tfidf', CountVectorizer()),
                ('bayes', BernoulliNB(alpha=0.3, force_alpha=True))])

In [ ]:
learner_lstm

Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=LstmModel(
    (lstm): LSTM(200, 250, batch_first=True)
    (fc): Linear(in_features=250, out_features=2, bias=True)
  ),
))])

In [ ]:
learner_bert

Pipeline(steps=[('tokenizer',
                 HuggingfacePretrainedTokenizer(max_length=255,
                                                tokenizer='bert-base-multilingual-cased')),
                ('bert',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=BertModel(
    (bert): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(119547, 768, padding_idx=0)
          (position_e...
                (dense): Linear(in_features=3072, out_features=768, bias=True)
                (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
        )
        (pooler): BertPooler(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (activation): Tanh()
        )
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (classifier): Linear(in_features=768, out_features=2, bias=True)
    )
  ),
))])

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_regression = LogisticRegression()

logistic_regression

LogisticRegression()

In [ ]:
dataset = Utils.read_csv_file('datasets/datasetall.csv')

CSV file read successfully!


In [ ]:
TEST_SIZE = 0.1

X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

In [ ]:
X_train

0        pag hindi nanalo si Norberto Gonzales pwede ba...
1        Ngayon lang ako super proud sa PRESIDENTE na i...
2        JUST SAW SOMEONE CALL BBM BLENGBLONG HAHAHAHAH...
3        Rep. Binay on her leadership style: I am very ...
4        Liwanag o dilim? May oras pa. Kakampink Leni L...
                               ...                        
25611    "Kala ko wala andito pala si Marcos."*pertaini...
25612    cathy [USERNAME] Dec [USERNAME] parang tanga i...
25613                             Nognog+pandak= BINAY ftw
25614    BINAY:Did your enormous wealth all come from y...
25615                                Uunlad tayo kay Binay
Name: text, Length: 25616, dtype: object

In [ ]:
y_train

0        1
1        0
2        1
3        0
4        0
        ..
25611    0
25612    1
25613    1
25614    1
25615    0
Name: label, Length: 25616, dtype: int64

In [ ]:
from src import EnsembleFunctions

predictions = EnsembleFunctions.concatenate_predict_proba(
  learner_bayes,
  learner_lstm,
  learner_bert,
  X_train,
)

predictions

KeyboardInterrupt: 

In [ ]:
from sklearn.ensemble import StackingClassifier

ensemble = StackingClassifier(
    estimators=[
        ('bayes', learner_bayes),
        ('lstm', learner_lstm),
        ('bert', learner_bert),
    ],
    final_estimator=logistic_regression,
    cv='prefit',
)

ensemble

StackingClassifier(cv='prefit',
                   estimators=[('bayes',
                                Pipeline(steps=[('tfidf', CountVectorizer()),
                                                ('bayes',
                                                 BernoulliNB(alpha=0.3,
                                                             force_alpha=True))])),
                               ('lstm',
                                Pipeline(steps=[('tokenizer',
                                                 CalamancyTokenizer()),
                                                ('lstm',
                                                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=LstmModel(
    (lstm): LSTM(200, 250, batch_first=True)
    (fc): Linear(in_feat...
                (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
            )
          )
        )
        (pooler): BertPooler(
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (activation): Tanh()
        )
      )
      (dropout): Dropout(p=0.1, inplace=False)
      (classifier): Linear(in_features=768, out_features=2, bias=True)
    )
  ),
))]))],
                   final_estimator=LogisticRegression())

In [ ]:
dataset = Utils.read_csv_file('datasets/datasetall.csv')

dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


In [ ]:
ensemble.fit(X_train, y_train)

In [ ]:
soft_voting_results_discrete = np.argmax(soft_voting_results, axis=1)

soft_voting_results_discrete

array([1, 1, 0, ..., 1, 1, 0])

In [ ]:
pd.DataFrame([{
  'accuracy': accuracy_score(soft_voting_results_discrete, y_test),
  'recall': recall_score(soft_voting_results_discrete, y_test),
  'precision': precision_score(soft_voting_results_discrete, y_test),
  'f1': f1_score(soft_voting_results_discrete, y_test),
}])

,accuracy,recall,precision,f1
0,0.846397,0.821821,0.887727,0.853503


In [ ]:
stacking_results = EnsembleFunctions.stacking(
  learner_bayes,
  learner_lstm,
  learner_bert,
  learner_lr,
  X_test
)

stacking_results

array([[0.0935605 , 0.9064395 ],
       [0.07668313, 0.92331687],
       [0.98238279, 0.01761721],
       ...,
       [0.05485927, 0.94514073],
       [0.05103099, 0.94896901],
       [0.96259484, 0.03740516]])

In [ ]:
stacking_results_discrete = np.argmax(stacking_results, axis=1)

stacking_results_discrete

array([1, 1, 0, ..., 1, 1, 0])

In [ ]:
pd.DataFrame([{
  'accuracy': accuracy_score(stacking_results_discrete, y_test),
  'recall': recall_score(stacking_results_discrete, y_test),
  'precision': precision_score(stacking_results_discrete, y_test),
  'f1': f1_score(stacking_results_discrete, y_test),
}])

,accuracy,recall,precision,f1
0,0.849912,0.846524,0.857741,0.852096
